In [8]:
import json
import requests
import urllib.parse
base_url = 'https://itunes.apple.com/search'

artist = 'Miyagi'

params = {
    'term': artist,
    'limit': 50 ,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [9]:
# Все доступные ключи объекта
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [10]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)

# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))


['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [11]:
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

In [12]:
print(set(map(lambda x: x['artistName'], data)))
Count = lambda x: x['trackName']
Count_Of_Songs = len(list(map(Count, data)))

print('Количество песен:' ,Count_Of_Songs)


print(set(list(map(lambda x: x['primaryGenreName'], data))))

Album = lambda x: x['collectionId']
Album_count = len(list(set(map(Album, data))))
print('Количество альбомов:', Album_count);

time_extractor = lambda x: x['trackTimeMillis']

Duration_of_songs = sum(list(map(time_extractor, data)))
print('Длительность всех песен:', convert_millis(Duration_of_songs))

Average_duration = Duration_of_songs / Count_Of_Songs
print('Средняя длительность песен:', convert_millis(Average_duration))

Median_time = list(map(time_extractor, data))
import statistics
print('Медиана длительности песен:', convert_millis(statistics.median(Median_time)))

longest_song = max(data, key=time_extractor)
print('Самая длинная песня:', longest_song['trackName'], ' и её длительность: ', convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)
print('Самая короткая песня:', shortest_song['trackName'], ' и её длительность: ', convert_millis(shortest_song['trackTimeMillis']));
Song_cost = lambda x: x['trackPrice']

All_price = sum(list(map(Song_cost, data)))
print('Стоимость всех песен:', All_price, '$')

Average_price = All_price / Count_Of_Songs
print('Средняя стоимость песен:', Average_price, '$')

Median_cost = list(map(Song_cost, data))
import statistics
print('Медиана стоимости песен:', statistics.median(Median_cost), '$')

Expensive_song = max(data, key=Song_cost)
print('Самая дорогая песня:', Expensive_song['trackName'])
print( 'стоимость самой дорогой песни:',  Expensive_song['trackPrice'], '$')

Cheap_song = min(data, key=Song_cost) 
print('Самая дешёвая песня:', Cheap_song['trackName'],'Free')
print('Самая дешёвая песня:', Cheap_song['trackPrice'], '$')

{'al l bo, Wooshendoo & Miyagi & Эндшпиль', 'Miyagi', 'Miyagi & Эндшпиль', 'Miyagi & Эндшпиль & NERAK', 'Andy Panda, Скриптонит, 104, TumaniYO, Miyagi', 'kavabanga Depo kolibri & Miyagi'}
Количество песен: 50
{'Hip-Hop/Rap', 'Reggae', 'New Age', 'Urbano latino', 'House'}
Количество альбомов: 20
Длительность всех песен: 3H14M56S
Средняя длительность песен: 0.0H3.0M53.0S
Медиана длительности песен: 0.0H3.0M42.0S
Самая длинная песня: Temple of Four Winds  и её длительность:  0H6M44S
Самая короткая песня: DLBM  и её длительность:  0H2M33S
Стоимость всех песен: 62.99999999999998 $
Средняя стоимость песен: 1.2599999999999996 $
Медиана стоимости песен: 1.29 $
Самая дорогая песня: I Got Love (feat. Рем Дигга)
стоимость самой дорогой песни: 1.29 $
Самая дешёвая песня: Mantra Rhythm Free
Самая дешёвая песня: 0.99 $


In [13]:
album_count = len( list( set( map( lambda x: x['collectionName'], data ) ) ) );

for i in range(album_count):
    album_name = list( set( map( lambda x: x['collectionName'], data)))[i]
    album_data = list( filter( lambda x:album_name in x['collectionName'], data))

    
    print( 'Название альбома:', set(list( map( lambda x: x['collectionName'], album_data))))
    print( 'Количество песен:', len( list( map( lambda x: x['trackName'], album_data))))
    print( 'Самый популярный жанр:',  set(list( map( lambda x: x['primaryGenreName'], album_data))))
    print( 'Дата релиза альбома:',  list( map( lambda x: x['releaseDate'], album_data))[0])
    
    longest_song = max(data, key=time_extractor)
    print('Самая длинная песня:', longest_song['trackName'], ' и её длительность: ', convert_millis(longest_song['trackTimeMillis']))
    shortest_song = min(data, key=time_extractor)
    print('Самая короткая песня:', shortest_song['trackName'], ' и её длительность: ', convert_millis(shortest_song['trackTimeMillis']))
    
    print( 'Общая длительность песен:', convert_millis( sum( list( map( lambda x: x['trackTimeMillis'], album_data)))))
    print( 'Средняя длительность песен:', convert_millis( round( statistics.mean( list( map( lambda x : x['trackTimeMillis'], album_data))))))
    print( 'Медиана длительности песен:', convert_millis( round( statistics.median( list( map( lambda x : x['trackTimeMillis'], album_data))))))
    
    print( 'общая стоимость песен: ≈', round( sum( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ), '$' )
    print( 'средняя стоимость песен: ≈', round( statistics.mean( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ), '$' )
    print( 'медиана стоимости песен: ≈', round( statistics.median( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ), '$' )
    print( 'название самой дорогой песни:', max( album_data, key = lambda x: x['trackPrice'] )['trackName'] )
    print( 'стоимость самой дорогой песни:', max( album_data, key = lambda x: x['trackPrice'] )['trackPrice'], '$' )
    print( 'название самой дешевой песни:', min( album_data, key = lambda x: x['trackPrice'] )['trackName'] )
    print( 'стоимость самой дешевой песни:', min( album_data, key = lambda x: x['trackPrice'] )['trackPrice'], '$' )
    print( '------------------------------------' )

Название альбома: {'Tai Chi'}
Количество песен: 5
Самый популярный жанр: {'New Age'}
Дата релиза альбома: 2001-04-03 12:00:00+00:00
Самая длинная песня: Temple of Four Winds  и её длительность:  0H6M44S
Самая короткая песня: DLBM  и её длительность:  0H2M33S
Общая длительность песен: 0H26M14S
Средняя длительность песен: 0H5M14S
Медиана длительности песен: 0H4M50S
общая стоимость песен: ≈ 4.95 $
средняя стоимость песен: ≈ 0.99 $
медиана стоимости песен: ≈ 0.99 $
название самой дорогой песни: Mantra Rhythm
стоимость самой дорогой песни: 0.99 $
название самой дешевой песни: Mantra Rhythm
стоимость самой дешевой песни: 0.99 $
------------------------------------
Название альбома: {'Сонная лощина - Single'}
Количество песен: 1
Самый популярный жанр: {'Hip-Hop/Rap'}
Дата релиза альбома: 2018-05-18 12:00:00+00:00
Самая длинная песня: Temple of Four Winds  и её длительность:  0H6M44S
Самая короткая песня: DLBM  и её длительность:  0H2M33S
Общая длительность песен: 0H3M33S
Средняя длительность 